In [ ]:
#### Load Libraries
import numpy as np
from joblib import load
from keras.models import Model, load_model
import glob 
import  pandas as pd
import matplotlib.pyplot as plt
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
import os
from pymongo import MongoClient
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
from datetime import timedelta

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
#####DB Config
host= "DevEdgeV32"
port=27017

In [ ]:
### set parameters
preprocessor_dir='G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/preprocessor/'
preprocessor_name='preprocessor.p'
model_dir='G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/model/'
model_name= 'model.h5'
selected_Columns=  ['t_bett','t_motor', 't_spindle','M8', 'M121', 'M127', 'M7']# So must be the input of the model
output_variable= 'welle_z'
prediction_variable= 'prediction'
window=10
shift= 1
time_shift= +1
all_data= True

In [ ]:
start= datetime.strptime("2022-12-09T00:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
end= datetime.strptime("2022-12-12T12:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000") 

Read Data From the machine

In [ ]:
## Show the input of the model from db
client = MongoClient(host= host, port=port)
db = client.h4ai
event_list = db.modelLogs.find({ "date" : { '$gte' : start, '$lt' : end} }).sort('date', 1)
signals= None
i=0
last_prediction= -500
### Go through events in DB
for event in event_list:
    i+=1
    record_list= event['content']
    keys= None
    ### Go through records in each event
    for record in record_list:
        if record['date']< start or record['date'] > end:
            continue
        if not all_data and not record['given2model']:
            continue
            # if 'given2model' not in record:
            #     if record[prediction_variable] == last_prediction:
            #         record['given2model']= False
            #     else:
            #         last_prediction= record[prediction_variable]
            #         record['given2model']= True
        if 'given2model' in record:
           record['raw_data']['given2model'] =record['given2model']
        keys= record['raw_data'].keys()
        for item in keys:
            record['raw_data'][item]= [record['raw_data'][item]]
        #print ('before',record)
        record['raw_data']['date']=[record['date'] + timedelta(hours= time_shift)]
        record['raw_data'][output_variable]=[record[output_variable]]
        record['raw_data'][prediction_variable]=[record[prediction_variable]]
        record['raw_data']['DRZ2'][0]= float(record['raw_data']['DRZ2'][0])
        # record['raw_data']['M8']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['M121']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['M127']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['M7']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['t_spindle'][0]= np.round(record['raw_data']['t_spindle'][0] - record['raw_data']['t_bett'][0], decimals=2)
        # record['raw_data']['t_motor']= np.round(float(record['raw_data']['t_motor'][0]) - record['raw_data']['t_bett'][0], decimals= 2)
        record['raw_data']['t_spindle'][0]= np.round(record['raw_data']['t_spindle'][0], decimals=2)
        record['raw_data']['t_motor']= np.round(float(record['raw_data']['t_motor'][0]), decimals= 2)
        #print ('After ',record)
        lf_signal_point= pd.DataFrame(record['raw_data'])
        if signals is None:
            signals= lf_signal_point
        else:
            signals= signals.append(lf_signal_point,ignore_index= True)
signals['t_motor']= signals['t_motor'].apply(np.float32)
# signals[prediction_variable]= signals[prediction_variable].shift(+1)
# signals= signals[1:]
#signals = remove_peeks(signals, output_variable, 4)
#signals.rename(columns={'t_spindle': 't_spindel', 'DRZ': 'drhz'}, inplace= True)

Filtering Data

In [ ]:
signals[output_variable].mask(1000*signals[output_variable]>60, 0, inplace= True)

In [ ]:
scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
#print(preds)
prediction_error= 1000*(signals[output_variable] - signals[prediction_variable])
fig= make_subplots(rows=11,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)

##t_bett
fig.add_trace(go.Scatter(x= signals['date'], y= signals['t_bett'], name='t_bett', mode= scatter_mode), row= 1, col= 1)
fig.update_yaxes(title_text= 't_bett C°', row= 1, col= 1)
##t_motor
fig.add_trace(go.Scatter(x= signals['date'], y= signals['t_motor'], name='t_motor', mode= scatter_mode), row= 2, col= 1)
fig.update_yaxes(title_text= 't_motor C°', row= 2, col= 1)
##t_spindel
fig.add_trace(go.Scatter(x= signals['date'], y= signals['t_spindle'], name='t_spindel', mode= scatter_mode), row= 3, col= 1)
fig.update_yaxes(title_text= 't_spindel C°', row= 3, col= 1)
###M8
fig.add_trace(go.Line(x= signals['date'], y=signals['M8'], name='M8'), row= 4, col=1)
fig.update_yaxes(title_text='M8', row= 4, col=1)
###M121
fig.add_trace(go.Line(x= signals['date'], y=signals['M121'], name='M121'), row= 5, col=1)
fig.update_yaxes(title_text='M121', row= 5, col=1)
###M127
fig.add_trace(go.Line(x= signals['date'], y=signals['M127'], name='M127'), row= 6, col=1)
fig.update_yaxes(title_text='M127', row= 6, col=1)
###M7
fig.add_trace(go.Line(x= signals['date'], y=signals['M7'], name='M7'), row= 7, col=1)
fig.update_yaxes(title_text='M7', row= 7, col=1)
##DRZ2
fig.add_trace(go.Scatter(x= signals['date'], y= signals['DRZ2'], name='DRZ2', mode= scatter_mode), row= 8, col= 1)
fig.update_yaxes(title_text= 'DRZ2', row= 8, col= 1)
##'given2model'
fig.add_trace(go.Scatter(x= signals['date'], y= signals['given2model'], name='given2model', mode= scatter_mode), row= 9, col= 1)
fig.update_yaxes(title_text= 'given2model 0/1', row= 9, col= 1)
##Verlagerung
fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[output_variable], name='Gemesene Verlagerung (Welle)', mode= scatter_mode), row= 10, col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[prediction_variable], name='KI-basierte Verlagerung (Welle)', mode= scatter_mode), row= 10, col= 1)
fig.update_yaxes(title_text= 'Verlagerung Micro-Meter', row= 10, col= 1)
##Restfehler
fig.add_trace(go.Scatter(x= signals['date'], y= prediction_error, name=' Restfehler', mode= scatter_mode),  row= 11 , col= 1)
## Draw the tolerence +-5
fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(prediction_error,5), name='+5 Obere Grenze', mode= scatter_mode),  row= 11 , col= 1)
fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(prediction_error,-5), name='- 5 Obere Grenze', mode= scatter_mode),  row= 11 , col= 1)
fig.update_yaxes(title_text= output_variable, row= 11, col= 1)
fig.update_layout(height=1200, width=1200, title_text= 'M57002 Machine Data')
fig.show()